In [1]:
!pip install mo-gymnasium
!pip install git+https://github.com/LucasAlegre/morl-baselines.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 950.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.6/479.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
  Cloning https://github.com/LucasAlegre/morl-baselines.git to /tmp/pip-req-build-kw98m2cn
  Running command git clone --filter=blob:none --quiet https://github.com/LucasAlegre/morl-baselines.git /tmp/pip-req-build-kw98m2cn
  Resolved https://github.com/LucasAlegre/morl-baselines.git to commit 92572aa5ee13e98b9e080f64df8e29602dba4f69
  Installi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import gymnasium as gym
import mo_gymnasium as mo_gym
import numpy as np
import sys
sys.path.append('/content/drive/MyDrive')
from mo_awr.MO_AWR import MO_AWR

# Deep Sea Treasure

A standard deterministic MOMDP which we can use to test the algorithm initially.

In [2]:
env = mo_gym.make("deep-sea-treasure-v0")
agent = MO_AWR(env, log=False, td_lambda=.8, use_popf=True, max_buffer_size=1024, value_lr=1e-4, policy_lr=5e-4, popf_lr=1e-4, batch_size=4, beta=1, use_is_weighting=True)

agent.train(1000000, 512, num_value_steps=500, num_policy_steps=500, num_popf_steps=500, num_pf_points=10, num_expl_episodes=64, log_every=1, prune_pf_every=15, pf_prune_threshold=np.array([1,5]))

/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


Pareto Front: [[array([ 0.618, -2.009], dtype=float32) 20.0]
 [array([ 0.231, -0.516], dtype=float32) 2.0]
 [array([ 0.252, -0.65 ], dtype=float32) 3.0]
 [array([ 0.903, -2.42 ], dtype=float32) 48.0]
 [array([ 0.978, -2.498], dtype=float32) 62.0]
 [array([ 0.294, -0.906], dtype=float32) 5.0]
 [array([ 0.482, -1.398], dtype=float32) 8.0]
 [array([ 0.416, -0.93 ], dtype=float32) 3.0]
 [array([ 0.212, -0.387], dtype=float32) 1.0]
 [array([ 1.043, -2.554], dtype=float32) 81.0]]
step 7313 	 return [ 0.841 -6.292], ([0.341 1.667]), new return [ 0.832 -2.009] 	 value loss 1.252E+01 	 policy loss 1.975E+00 	 popf loss 1.049E+00
Pareto Front: [[array([ 0.429, -0.933], dtype=float32) 2.0]
 [array([ 0.252, -0.65 ], dtype=float32) 3.0]
 [array([ 1.197, -2.864], dtype=float32) 11.0]
 [array([ 0.363, -0.677], dtype=float32) 1.0]
 [array([ 0.562, -1.437], dtype=float32) 4.0]
 [array([ 0.496, -1.191], dtype=float32) 3.0]
 [array([ 0.845, -1.826], dtype=float32) 4.0]
 [array([ 0.624, -1.666], dtype=flo

KeyboardInterrupt: 

# Random MOMDP

In order to test MO-AWR on a random environment, we can use the code at https://github.com/rradules/POP-following/blob/main/envs/randommomdp.py to generate an MOMDP where we can decide randomness and the problem size ourselves.



In [5]:
env = mo_gym.make("random_momdp-v0", nstates=5, nobjectives=2, nactions=3, nsuccessor=2, seed=12)

from gymnasium.wrappers import FlattenObservation
env = FlattenObservation(env)

agent = MO_AWR(env, log=False, td_lambda=.9, use_popf=True, max_buffer_size=1024, value_lr=1e-4, policy_lr=1e-4, popf_lr=1e-4, batch_size=8, beta=1)

agent.train(1000000, 512, num_value_steps=500, num_policy_steps=500, num_popf_steps=500, num_pf_points=10, num_expl_episodes=64, log_every=1, prune_pf_every=25, num_eval_iter=10, pf_prune_threshold=np.array([5,5]))

/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


Pareto Front: [[array([18.667, 26.086]) 50.0]
 [array([19.396, 25.967]) 50.0]
 [array([17.409, 26.533]) 50.0]]
step 30300 	 return [6.211 9.348], ([0.464 0.505]), new return [18.667 26.303] 	 value loss 2.067E+01 	 policy loss 1.800E+00 	 popf loss 3.868E+00


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/mo_awr/Results/data/points_1.csv'

# Water Reservoir (discrete)
A discrete version of a dam MOMDP (https://github.com/Farama-Foundation/MO-Gymnasium/blob/main/mo_gymnasium/envs/water_reservoir/dam_env.py).



In [3]:
env = mo_gym.make("dam_env-v0", s_0=np.int64([5]), seed=0, penalize=False)
agent = MO_AWR(env, log=False, td_lambda=.75, use_popf=True, max_buffer_size=1024, value_lr=1e-4, policy_lr=5e-4, popf_lr=1e-4, batch_size=8, beta=1, use_is_weighting=True)
agent.train(1000000, 512, num_value_steps=500, num_policy_steps=500, num_popf_steps=500, num_pf_points=10, num_expl_episodes=64, log_every=1, prune_pf_every=25, num_eval_iter=20, pf_prune_threshold=np.array([10,10]))

/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


AttributeError: 'Dam' object has no attribute 'power_demand'

In [5]:
from morl_baselines.multi_policy.pareto_q_learning.pql import PQL
pql_agent = PQL(env, np.array([-100, -100]), gamma=1)
pql_agent.train(
    500000,
    env,
    num_eval_weights_for_eval=10,
)


1

# Buridan's Ass

This MOMDP version of the philosophical dilemma shows a donkey in a 3x3 grid. It starts at the center square right in between 2 food piles. Whenever it moves away from an adjacent food pile, the pile ahs a probability p_stolen to get stolen. Food is regenerated after n_appear steps.

States are tuples (s,f,t):
- s indicates grid cell from 0 to 8
- f indicates food in both piles as a binary tuple
- t indicates time since the donkey last ate

Actions are discrete:
- 0: move up
- 1: move down
- 2: move left
- 3: move right
- 4: wait

Rewards are 3-dimensional:
- Hunger penalty: -1 if not eaten for max_time_hungry steps
- Food stolen penalty: -.5 for each stolen pile of food
- Walking penalty: -1 for each step taken, 0 if waiting

We  will try to find the policies which at least try to maximize either reward. However, minimizing the walking penalty and the food stolen penalty do lead to similar policies.


In [1]:
from gymnasium.wrappers import FlattenObservation
env = mo_gym.make("buridans_ass-v0", p_stolen=.9, n_appear=5, max_steps=50)
env = FlattenObservation(env)
agent = MO_AWR(env, log=False, td_lambda=.9, use_popf=True, max_buffer_size=1024, value_lr=5e-5, policy_lr=1e-4, popf_lr=1e-4, batch_size=4, beta=.1, use_critic_msbe=False)
agent.train(1000000, 512, num_value_steps=500, num_policy_steps=1000, num_popf_steps=1000, num_pf_points=5, num_expl_episodes=64, log_every=1, prune_pf_every=30, num_eval_iter=25, pf_prune_threshold=np.array([10,10,10]))

NameError: name 'mo_gym' is not defined

In [3]:
!zip -r /content/drive/MyDrive/mo_awr/results.zip /content/drive/MyDrive/mo_awr/Results/

  adding: content/drive/MyDrive/mo_awr/Results/ (stored 0%)
  adding: content/drive/MyDrive/mo_awr/Results/plots/ (stored 0%)
  adding: content/drive/MyDrive/mo_awr/Results/plots/training_plot_1.png (deflated 13%)
  adding: content/drive/MyDrive/mo_awr/Results/plots/training_plot_2.png (deflated 13%)
  adding: content/drive/MyDrive/mo_awr/Results/plots/training_plot_3.png (deflated 12%)
  adding: content/drive/MyDrive/mo_awr/Results/plots/training_plot_4.png (deflated 13%)
  adding: content/drive/MyDrive/mo_awr/Results/plots/training_plot_5.png (deflated 12%)
  adding: content/drive/MyDrive/mo_awr/Results/plots/training_plot_6.png (deflated 12%)
  adding: content/drive/MyDrive/mo_awr/Results/plots/training_plot_7.png (deflated 12%)
  adding: content/drive/MyDrive/mo_awr/Results/plots/training_plot_8.png (deflated 12%)
  adding: content/drive/MyDrive/mo_awr/Results/plots/training_plot_9.png (deflated 14%)
  adding: content/drive/MyDrive/mo_awr/Results/plots/training_plot_10.png (deflate

In [6]:
!rm -rf /content/drive/MyDrive/mo_awr/Results/

In [ ]:
drive.flush_and_unmount()

In [5]:
agent.experience_replay

Output hidden; open in https://colab.research.google.com to view.

# Training Notes
- Most episodes in the ER only reach the first reward:
  - ER buffer is constructed such that new episodes are always added
  - This removes episodes by:
    1. Duplicates
    2. Distance metric (L2 + crowding)
    3. Oldest first
  - better returns are achieved as better PF returns are added to the PF

- buffer size, batch size and num expl episodes are important choices:
  - Num expl episodes needs to add enough episodes for them to be of significance during the next training iteration(s)
  - If num_exl is too large (and buffer size too small) then exploration episodes can overwrite episodes with good returns!
  - The smaller the batch size, the more these episodes have impact on training => faster convergence (depending on environment0
  - Larger buffer size means less chance of these exploration episodes to be used for training in the next timestep. A sufficient size is needed to ensure enough points are in te buffer for computing each point in the pf (at least batch_size*num_pf_points, but pref. more)
  - Preferably small batch size (16/32/64) and large buffer (1024/2048) depending on problem size
  - Depending on buffer size, pick sufficiently large number of exploration episodes to increase convergence speed

Idea:
- In order to improve the PF over time, average returns (from s0) must improve
- We need sufficient 'better' trajectories (at least batch_size) for this average to improve
- Take batches of (s,a) instead of only over state. This implies that taking similar actions in similar states leads to similar polices, which in term helps with convergence
- Using such batches could also solve loss/gradient issues!
- This complies with using this model for stochastic transitions, as we want averages over state action pairs
- This implies that we learn deterministic policies

Questions:
- Policy explorations improved drastically after using tanh inbetween hidden layers
- Value estimates are more accurate when using tanh
- Weighting losses using IS ratio should help because data is very off-policy compared to single-objective RL.
- Try to modify V such that:
  - V is conditioned on (s,a,R,h), with R being observed returns and outputs expected returns. This way, V learns the relation between returns and expected returns.
  - The value loss function should not be weighted this way
  - The TD(λ) return needs to be modified slightly => no keeping track of followed value vector, but rather keep track of the MC return (this way we can feed this return into V to get that part of the return)
- OR: try to decrease TD(λ) such that expectation is learned implicitly over time instead of by batching
- The MC return and the value function have a mismatch (observed vs expected returns), this can become a problem in random environments!
- The policy makes extra steps to get to some returns, even if it knows how to reach them

TODO:
- After policy evaluation it is clear that the agent often does not reach a terminal state => it learns to make cycles and uses these bootstrapped estimates to get its rewards...
  - weight policy loss so off-policy actions contribute less => less-likely to learn wrong actions by off policy learning... upon policy eval, the policies result in the first reward only
  - Experiment with Beta: slightly larger values result in better policy eval!

- Try to balance actor and critic updates:
  - In AWR, the actor is updated more than the critic. This is because the critic is only used to compute the advantage weights, and not for guiding the policy gradients themselves.
  - AWR remains stable without any target network for the critic for this same reason. In our case, value estimates are used for computing the TD(lambda) return (as in AWR). But this return is then used as conditioning for both the actor and critic.

- PF contains 'intermediate' returns: averages of different solutions which are non-dominated by themselves, but are not a distinct solution when evaluated.
  - This is problem specific...
  - Eg. the 3 returns next to each other are very close together (thus have a low crowding distance). Expected returns between 0.7 and 8.3 are in the PF, but not achievable (unless by stochastic policy). This is because crowding distance is higher than for the others.
  - We can replace the PF points by their evaluated returns (taking Pareto dominance into account).
    - ISSUE: mismatch between learned returns, PF and ER returns: picking a pf return and improving it by a slight amount can become impossible due to the policy possibly never having learned such returns... => try to add more expl episodes
    - Another issue is that we stop learning to condition on expected returns and replace them by learning from observations!
    - IDEA: evaluate policies multiple times, this way we can get expected return (important in stochastic env). Only keep PF returns when their evaluated return is close to itself (by some threshold for each objective)

- Learn a PCS for every state (this way we can handle multiple initial states) (difficult to plot though)

- When we want to compute the n-step return, we need to use the correct value vector to give to the value function:
  - If we use the one saved as value_vec (in the transitions), then we are not guaranteed to use the correct one. The value vector followed during exploration or a random policy does not match the actual value vector of that policy!
  - IDEA: use value function to learn relation between observed return and expected returns
    - use mean_r as targets
    - weight losses as before
    - This implies that each policy is uniquely identifiable by V as a pair of a desired return and a horizon
    - compute TD(λ) return from state s as: λ*MC return + (1-λ)V(s,R,h)
    - for policy and for advantage weight computation use V(s,R,h) instead of R_hat! This should loosen dependency between averaged returns and learned actions
    - The policy needs to be conditioned on expected returns => POPF needs to output correct vectors!

    